In [2]:
import pandas as pd
import numpy as np 
import matplotlib as mpl
import matplotlib.pyplot as plt
import pickle
import glob
import os

In [27]:
interfaces_test_data = pd.read_csv("/rcfs/projects/proteometer/human_proteome_precaculated/interfaces_precaculated.csv")
pockets_test_data = pd.read_csv("/rcfs/projects/proteometer/human_proteome_precaculated/pockets_precaculated.csv")
functions_test_data = pd.read_csv("/rcfs/projects/proteometer/human_proteome_precaculated/functions_precalculated.csv")

# don't need to merge stability data bc interfaces and pockets already contain it

/tmp/ipykernel_15507/1837385787.py:3: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  functions_test_data = pd.read_csv("/rcfs/projects/proteometer/human_proteome_precaculated/functions_precalculated.csv")


In [30]:
print(interfaces_test_data.shape)
print(pockets_test_data.shape)
print(functions_test_data.shape)

(8544123, 16)
(8544123, 17)
(8544123, 17)


In [31]:
sasa_test_data = pd.read_csv("/rcfs/projects/proteometer/human_proteome_precaculated/sasa_precaculated.csv")
charge_test_data = pd.read_csv("/rcfs/projects/proteometer/human_proteome_precaculated/charge_precaculated.csv")

In [32]:
print(sasa_test_data.shape)
print(charge_test_data.shape)


(10486334, 8)
(10486334, 8)


In [33]:
pka_test_data = pd.read_csv("/rcfs/projects/proteometer/human_proteome_precaculated/pka_precaculated.csv")


In [34]:
print(pka_test_data.shape)


(5916614, 7)


* there are 3 groups:
    * pockets, interfaces, stability, and functions data
    * sasa and charge
    * pka

In [26]:
print(interfaces_test_data['position'].dtype)
print(sasa_test_data['RES_NUM'].dtype)
print(pka_test_data['res_number'].dtype)
# making sure all of the residue num columns are the same type so they can actually be compared

int64
int64
int64


In [35]:
sasa_charge =  pd.merge(sasa_test_data, charge_test_data[['charge', 'charge_method', 'UNIPROT', 'RES_NUM']], how= 'inner', left_on=['UNIPROT', 'RES_NUM'], right_on=['UNIPROT', 'RES_NUM'])
print(sasa_charge.shape)
sasa_charge

(10486334, 10)


,Unnamed: 0.1,Unnamed: 0,all_sasa_value,atom_sasa_values,residue_number,residue_name,UNIPROT,RES_NUM,charge,charge_method
0,0,0,97.501263,"[34.99482888686743, 12.076282160399163, 1.2076...",1,MET,Q9UJT0,1,"[-0.12303608460739965, 0.17017601567027532, 0....",gasteiger
1,1,1,28.166062,"[0.0, 0.0, 0.0, 0.0, 2.142918048125441, 21.737...",2,THR,Q9UJT0,2,"[-0.19389656694946025, 0.20280639831432518, 0....",gasteiger
2,2,2,6.561530,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.56153041...",3,GLN,Q9UJT0,3,"[-0.1962555527195044, 0.1732649034690787, 0.26...",gasteiger
3,3,3,6.428754,"[0.0, 0.0, 0.0, 0.0, 0.0, 6.428754144376322]",4,SER,Q9UJT0,4,"[-0.1935755721777093, 0.20547497321995334, 0.2...",gasteiger
4,4,4,0.000000,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",5,VAL,Q9UJT0,5,"[-0.19626896253115445, 0.17270206677327354, 0....",gasteiger
...,...,...,...,...,...,...,...,...,...,...
10486329,729,729,174.475398,"[13.123060832575288, 4.830512864159665, 3.6228...",730,LYS,Q96TA0,730,"[-0.19634140753571544, 0.17276953361914518, 0....",gasteiger
10486330,730,730,91.609470,"[15.310237638004502, 7.245769296239498, 2.4152...",731,ALA,Q96TA0,731,"[-0.1963954321204338, 0.17134749320023535, 0.2...",gasteiger
10486331,731,731,118.820664,"[0.0, 9.66102572831933, 1.2076282160399163, 24...",732,PRO,Q96TA0,732,"[-0.2716029983948695, 0.1520513571091095, 0.24...",gasteiger
10486332,732,732,112.391910,"[0.0, 10.868653944359247, 2.4152564320798326, ...",733,PRO,Q96TA0,733,"[-0.27161169395248486, 0.1535560784715306, 0.2...",gasteiger


In [36]:
interface_pockets =  pd.merge(interfaces_test_data, pockets_test_data[['protein_acc', 'position', 'closest_pocket', 'inside_pocket','min_distance_from_pocket','mean_distance_from_pocket','distance_from_pocket']], how= 'inner', left_on=['protein_acc', 'position'], right_on=['protein_acc', 'position'])
interface_pockets_functions = pd.merge(interface_pockets, functions_test_data[['protein_acc', 'position', 'closest_binding_site', 'closest_binding_site_decription','closest_binding_site_min_distance','closest_active_site','closest_active_site_decription','closest_active_site_min_distance','closest_dna_binding_site','closest_dna_binding_site_min_distance']], how= 'inner', left_on=['protein_acc', 'position'], right_on=['protein_acc', 'position'])
print(interface_pockets_functions.shape)
interface_pockets_functions

(8544123, 29)


,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,protein_acc,wild_type,position,plddt,foldx_ddg_min,foldx_ddg_max,...,mean_distance_from_pocket,distance_from_pocket,closest_binding_site,closest_binding_site_decription,closest_binding_site_min_distance,closest_active_site,closest_active_site_decription,closest_active_site_min_distance,closest_dna_binding_site,closest_dna_binding_site_min_distance
0,0,0,1375293,0,Q11128,M,1,40.87,-0.074884,0.525290,...,29.388597,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1,1375294,1,Q11128,D,2,38.65,-0.649167,2.248420,...,26.619948,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,2,1375295,2,Q11128,P,3,45.62,0.394833,1.166880,...,23.282309,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,3,1375296,3,Q11128,L,4,46.03,-0.910266,2.077580,...,19.928173,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,4,1375297,4,Q11128,G,5,45.32,-1.027170,0.223936,...,17.670545,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8544118,168203,168203,3049667,480,Q8IYM9,C,494,94.49,-1.481650,1.400760,...,11.413381,NaN,125.0,"{'ligand': '""Zn(2+)""'}",79.608464,NaN,NaN,NaN,NaN,NaN
8544119,168204,168204,3049668,481,Q8IYM9,P,495,91.87,0.632578,2.546670,...,9.958481,NaN,125.0,"{'ligand': '""Zn(2+)""'}",76.282333,NaN,NaN,NaN,NaN,NaN
8544120,168205,168205,3049669,482,Q8IYM9,P,496,88.26,1.511810,3.470850,...,7.259745,0.0,125.0,"{'ligand': '""Zn(2+)""'}",74.055230,NaN,NaN,NaN,NaN,NaN
8544121,168206,168206,3049670,483,Q8IYM9,S,497,70.65,-1.193310,0.661215,...,8.284621,0.0,125.0,"{'ligand': '""Zn(2+)""'}",75.403493,NaN,NaN,NaN,NaN,NaN


In [37]:
sasa_charge_interface_pockets_functions = pd.merge(sasa_charge,interface_pockets_functions, how = 'outer', left_on=['UNIPROT', 'RES_NUM'], right_on=['protein_acc', 'position'])
print(sasa_charge_interface_pockets_functions.shape)
sasa_charge_interface_pockets_functions_pka = pd.merge(sasa_charge_interface_pockets_functions,pka_test_data, how = 'outer', left_on=['UNIPROT', 'RES_NUM'], right_on=['uniprotID', 'res_number'])
print(sasa_charge_interface_pockets_functions_pka.shape)
sasa_charge_interface_pockets_functions_pka

(11120413, 39)
(13044316, 46)


,Unnamed: 0.1_x,Unnamed: 0_x,all_sasa_value,atom_sasa_values,residue_number,residue_name,UNIPROT,RES_NUM,charge,charge_method,...,closest_active_site_min_distance,closest_dna_binding_site,closest_dna_binding_site_min_distance,Unnamed: 0,uniprotID,AA,res_number,pK,state,position_y
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1.0,A0A024R1R8-F1,SER,2.0,NaN,undefined,2.0
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,2.0,A0A024R1R8-F1,SER,3.0,NaN,undefined,3.0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,3.0,A0A024R1R8-F1,HIS,4.0,6.559955,undefined,4.0
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,4.0,A0A024R1R8-F1,GLU,5.0,3.926335,deprotonated,5.0
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,5.0,A0A024R1R8-F1,LYS,8.0,10.446961,protonated,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13044311,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13044312,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13044313,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13044314,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [38]:
sasa_charge_interface_pockets_functions_pka.to_csv("/rcfs/projects/proteometer/human_proteome_precaculated/all_combined/sasa_charge_interface_pockets_functions_pka.csv")